In [21]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: D:\Python_code\RR_Model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000014BB1A44B80>, origin='D:\\Python_code\\RR_Model\\src\\__init__.py', submodule_search_locations=['D:\\Python_code\\RR_Model\\src'])


In [23]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob
#from src.rollrate.forecast import forecast_report
from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"D:\Python_code\RR_Model\sql\uns2.sql"

df = load_data(sql_path)
#df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model\sql\uns2.sql
First 200 chars:
 select * from TBL_RR_UNSECURED2
Params: {}


D:\Python_code\RR_Model\src\db.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(final_sql, conn, params=bind_params)


DatabaseError: DPI-1080: connection was closed by ORA-2396

In [4]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.export_excel import export_transition_excel
from src.config import OUT_ROOT

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

# Đếm real vs fallback
count_real = 0
count_fb = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fb += 1
            else:
                count_real += 1

print(f"🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fb}")
print(f"🔢 Tổng             : {count_real + count_fb}")

# Xuất Excel
export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)



=== Tính transition matrices theo Product × MOB × Score ===
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=B)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=C)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=D)
⚠️ Có 7 hàng có tổng weight = 0: ['DPD30+', 'DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 7 hàng có tổng weight = 0: ['DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 6 hàng có tổng weight = 0: ['DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 5 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 

In [5]:


export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)


✅ Xuất file transition thành công → D:\Python_code\RR_Model\outputs\Transition_ByProduct_MOB_Score_2.xlsx


In [6]:
# matrices_by_mob đã build từ module transition
# df = raw data chứa DISB_DATE, CUTOFF, MOB, PRINCIPLE_OUTSTANDING, STATE...

from src.rollrate.forecast import forecast_all_vintages


In [7]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False
)


In [15]:
from src.rollrate.lifecycle import build_full_lifecycle_amount,aggregate_loss_to_product,tag_forecast_rows_amount,export_lifecycle_all_products_one_file, aggregate_to_product, add_del_metrics 

In [ ]:
# 1) Build lifecycle actual + forecast
df_lifecycle = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=29)

# 2) Tag forecast
df_lifecycle_tagged = tag_forecast_rows_amount(df_lifecycle, df)

# 3) DEL30/60/90
df_del = add_del_metrics(df_lifecycle_tagged, df)

# 4) Aggregate lên Product
df_del_prod = aggregate_to_product(df_del)

# 5) Pivot lifecycle DEL30+
df_del30_pivot = df_del_prod.pivot_table(
    index="VINTAGE_DATE",
    columns="MOB",
    values="DEL30_PCT"
).fillna(0)

df_del30_pivot.index.name = "Cohort"

# 6) Actual max MOB (để kẻ đường đỏ)
actual_info = (
    df.groupby("DISBURSAL_DATE")["MOB"]
    .max()
    .to_dict()
)

# 7) Export Excel đầy đủ format
export_heatmap(
    df_pivot=df_del30_pivot,
    df_del_prod=df_del_prod,
    actual_info=actual_info,
    filename="DEL30P_Lifecycle.xlsx"
)

print("🎉 Lifecycle DEL30+ đã xuất thành công!")


In [10]:
# 1) Build lifecycle (actual + forecast, amount-based)
df_lifecycle = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
)

# 2) Tag forecast rows
df_lifecycle_tagged = tag_forecast_rows_amount(
    df_lifecycle=df_lifecycle,
    df_raw=df,
)

# 3) Tính DEL30/60/90 amount + pct
df_del = add_del_metrics(
    df_lifecycle=df_lifecycle_tagged,
    df_raw=df,
)

# 4) Aggregate Product_Score → Product
df_del_prod = aggregate_to_product(df_del)

# 5) Max actual MOB per (Product, Vintage)
actual_info = (
    df.groupby(["PRODUCT_TYPE", "DISBURSAL_DATE"])["MOB"]
      .max()
      .to_dict()
)

# 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
export_lifecycle_all_products_one_file(
    df_del_prod=df_del_prod,
    actual_info=actual_info,
    filename="Lifecycle_All_Products.xlsx",
)


D:\Python_code\RR_Model\src\rollrate\lifecycle.py:285: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


✔ Export lifecycle multi-product thành công → Lifecycle_All_Products.xlsx


In [11]:
# 6) Export chung 1 file, nhiều sheet (mỗi Product × metric)
export_lifecycle_all_products_one_file(
    df_del_prod=df_del_prod,
    actual_info=actual_info,
    filename="Lifecycle_All_Products9.xlsx",
)


✔ Export lifecycle multi-product thành công → Lifecycle_All_Products9.xlsx


In [13]:
key = ("SALPIL", "A", pd.Timestamp("2023-01-01"))
fc_seg = results[key]    # {mob: Series(EAD)}


NameError: name 'results' is not defined

In [14]:
from src.rollrate.forecast import (
    marginal_loss_rate,
    cumulative_loss_rate
)

# Marginal
marginal = marginal_loss_rate(fc_seg)
print("Marginal Loss Rate:")
print(marginal)

# Cumulative
cumulative = cumulative_loss_rate(fc_seg)
print("\nCumulative Loss Rate:")
print(cumulative)


NameError: name 'fc_seg' is not defined

In [ ]:
import pandas as pd
from src.rollrate.forecast import plot_curve

df_marginal = pd.DataFrame({
    "MARGINAL_LOSS": marginal
})

plot_curve(df_marginal, ["MARGINAL_LOSS"], title="Marginal Loss Curve")


In [ ]:
df_cum = pd.DataFrame({
    "CUM_LOSS": cumulative
})

plot_curve(df_cum, ["CUM_LOSS"], title="Cumulative Loss Curve")


In [19]:
loss_salp = aggregate_loss_to_product(
    forecast_results=forecast_results,
    df_raw=df,
    product_filter="SALPIL"
)


In [20]:
df_plot = pd.DataFrame({"LOSS_RATE": loss_curve})
plot_curve(df_plot, ["LOSS_RATE"], "Loss Curve SALPIL – 2023-01-01")


NameError: name 'loss_curve' is not defined

In [ ]:
pivot_loss = pd.DataFrame(loss_prod).T.reset_index()


In [24]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("output.parquet", compression="snappy", index=False)
